### Passo 1 - Importar Arquivos e Bibliotecas

In [32]:
#importar bibliotecas
import pandas as pd
import os
import pathlib
import win32com.client as win32
import shutil

In [33]:
vendas_df = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
emails_df = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas_df = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='ISO-8859-1', sep=';')

def substituir_espacos_por_underscore(s):
    return s.replace(" ", "_")

lojas_df['Loja'] = lojas_df['Loja'].apply(substituir_espacos_por_underscore)
emails_df['Loja'] = emails_df['Loja'].apply(substituir_espacos_por_underscore)

vendas_df = vendas_df.merge(lojas_df, on='ID Loja')

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [34]:
dicionario_lojas = {}
for loja in lojas_df['Loja']:
    dicionario_lojas[loja] = vendas_df.loc[vendas_df['Loja'] == loja, :]

display(dicionario_lojas['Shopping_Recife'])

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
32491,11,2019-01-02,18,Camiseta,1,180,180,Shopping_Recife
32492,11,2019-01-02,18,Calça Listrado,2,181,362,Shopping_Recife
32493,11,2019-01-02,18,Camisa Estampa,1,113,113,Shopping_Recife
32494,45,2019-01-02,18,Camisa Linho,1,141,141,Shopping_Recife
32495,96,2019-01-02,18,Meia Xadrez,1,41,41,Shopping_Recife
...,...,...,...,...,...,...,...,...
36377,69925,2019-12-25,18,Gorro Liso,2,92,184,Shopping_Recife
36378,69936,2019-12-25,18,Terno Linho,1,750,750,Shopping_Recife
36379,69971,2019-12-26,18,Cinto Liso,2,209,418,Shopping_Recife
36380,69971,2019-12-26,18,Bermuda Listrado,2,163,326,Shopping_Recife


In [35]:
#Dia do indicador
dia_indicador = vendas_df['Data'].max()
print(f'{dia_indicador.day}/{dia_indicador.month}')

26/12


### Passo 3 - Salvar a planilha na pasta de backup

In [36]:
#Identificar se a pasta já existe

caminho_backup = pathlib.Path(r'Backup_Arquivos_Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()
    
    #Salvar o arquivo atualizado dentro da pasta
    nome_do_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_do_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo)

In [53]:
#Movendo os arquivos de pasta 

#for loja in lojas_df['Loja']:
#    origem = f'C:\\Users\\rock_\\OneDrive\\Área de Trabalho\\Backup\\HASHTAG\\PROJETOS PYTHON\\Módulo 39 - Automações de Processo - Aplicação de Mercado de Trabalho\\Projeto AutomacaoIndicadores\\{loja}'
#    destino = f'C:\\Users\\rock_\\OneDrive\\Área de Trabalho\\Backup\HASHTAG\\PROJETOS PYTHON\\Módulo 39 - Automações de Processo - Aplicação de Mercado de Trabalho\\Projeto AutomacaoIndicadores\\Backup Arquivos Lojas\\{loja}'
#
#    try:
#        shutil.move(origem, destino)
#        print(f'A pasta {origem} foi movida para {destino}')
#    except Exception as e:
#        print(f"Erro ao mover a pasta: {e}")

### Passo 4 - Calcular o indicador para 1 loja

In [37]:
#Definição de metas

meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000

meta_diversidade_produto_dia = 4
meta_diversidade_produto_ano = 120

meta_ticket_medio_dia = 500
meta_ticket_medio_ano = 500

### Passo 5 - Enviar por e-mail para o gerente

In [ ]:
for loja in lojas_df['Loja']:
#for loja in dicionario_lojas:

    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data'] == dia_indicador,:]

    #faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    #diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    #ticket medio

    tabela_media = vendas_loja.groupby('Código Venda').sum(numeric_only=True)
    tabela_media_dia = vendas_loja_dia.groupby('Código Venda').sum(numeric_only=True)

    ticket_medio_ano = tabela_media['Valor Final'].mean()
    ticket_medio_dia = tabela_media_dia['Valor Final'].mean()

    #Definindo as cores dos indicadores do dia
    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'

    if qtde_produtos_dia >= meta_diversidade_produto_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'

    if ticket_medio_dia >= meta_ticket_medio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'

    #Definindo as cores dos indicadores do ano
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'

    if qtde_produtos_ano >= meta_diversidade_produto_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'

    if ticket_medio_ano >= meta_ticket_medio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'
    


#Envio do E-mail para todos os gerentes
    outlook = win32.Dispatch('outlook.application')

    nome = emails_df.loc[emails_df['Loja'] == loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails_df.loc[emails_df['Loja'] == loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

    mail.HTMLBody = f'''
    <p>Bom dia, {nome}</p>

    <p>O resultado de ontem (dia <strong>{dia_indicador.day}/{dia_indicador.month}</strong>) da Loja <strong>{loja}</strong> foi:</p>

    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
    </tr>
    <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_diversidade_produto_dia}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
    </tr>
    <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticket_medio_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
    </tr>
    </table>
    <br>
    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
    </tr>
    <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_diversidade_produto_ano}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
    </tr>
    <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticket_medio_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
    </tr>
    </table>
    <br>
    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida, estou à disposição.</p>

    <p>At.</p>

    <p>Erick Fernandes</p>
    '''

    # Anexos (pode colocar quantos quiser):
    attachment  = pathlib.Path.cwd() / caminho_backup / loja / nome_do_arquivo
   
    if os.path.exists(attachment):
        mail.Attachments.Add(attachment)
        mail.Send()
        print(f'E-mail da Loja {loja} enviado com sucesso!')
    else:
        print(f'O arquivo {nome_do_arquivo} não foi encontrado.')

    #attachment = fr'C:/Users/rock_/Desktop/HASHTAG/PROJETOS_PYTHON/Modulo39_Automacoes_de_Processo/Backup_Arquivos_Lojas/{loja}/{nome_do_arquivo}'
    # nome_do_arquivo = '12_26_Iguatemi_Campinas.xlsx'
    # attachment = 'C:/%s' %nome_do_arquivo
    #attachment_2 = 'C:\Users\rock_\Desktop\HASHTAG\PROJETOS_PYTHON\Modulo39_Automacoes_de_Processo\Backup_Arquivos_Lojas\Iguatemi_Esplanada\12_26_Iguatemi_Esplanada.xlsx'
    #attachment = 'C:/HASHTAG/PROJETOS_PYTHON/Modulo39_Automacoes_de_Processo/Backup_Arquivos_Lojas/%s/%s' %(loja, nome_do_arquivo)
    #attachment = attachment.replace("/", "\\")

### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

In [45]:
faturamento_lojas = vendas_df.groupby('Loja')[['Valor Final']].sum(numeric_only=True)
faturamento_lojas = faturamento_lojas.sort_values(by='Valor Final', ascending=False)

nome_do_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
faturamento_lojas.to_excel(r'Backup_Arquivos_Lojas\{}'.format(nome_do_arquivo))

vendas_df_dia = vendas_df.loc[vendas_df['Data'] == dia_indicador , :]
faturamento_lojas_dia = vendas_df_dia.groupby('Loja')[['Valor Final']].sum(numeric_only=True)
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)

nome_do_arquivo = '{}_{}_Ranking Diario.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
faturamento_lojas.to_excel(r'Backup_Arquivos_Lojas\{}'.format(nome_do_arquivo))


,Valor Final
Loja,
Iguatemi_Campinas,1762419
Shopping_Vila_Velha,1731167
Bourbon_Shopping_SP,1726110
Rio_Mar_Recife,1722766
Shopping_SP_Market,1721763
Palladium_Shopping_Curitiba,1721120
Norte_Shopping,1711968
Ribeirão_Shopping,1707122
Iguatemi_Esplanada,1699681


,Valor Final
Loja,
Salvador_Shopping,3950
Novo_Shopping_Ribeirão_Preto,3400
Center_Shopping_Uberlândia,2651
Shopping_Eldorado,2391
Shopping_Center_Interlagos,1582
Shopping_Recife,1366
Norte_Shopping,1259
Shopping_União_de_Osasco,1207
Shopping_Vila_Velha,937


### Passo 8 - Enviar e-mail para diretoria

In [46]:
#Envio do E-mail para todos os gerentes
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails_df.loc[emails_df['Loja'] == 'Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
mail.Body = f'''
Prezados, bom dia!

Melhor loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[0]} com Faturamento R$ {faturamento_lojas_dia.iloc[0,0]:.2f}.
Pior loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[-1]} com Faturamento R$ {faturamento_lojas_dia.iloc[-1,0]:.2f}.

Melhor loja do Ano em Faturamento: Loja {faturamento_lojas.index[0]} com Faturamento R$ {faturamento_lojas.iloc[0,0]:.2f}.
Pior loja do Ano em Faturamento: Loja {faturamento_lojas.index[-1]} com Faturamento R$ {faturamento_lojas.iloc[-1,0]:.2f}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou a disposição.

At.

Erick Fernandes

'''

# Anexos (pode colocar quantos quiser):
attachment  = pathlib.Path.cwd() / caminho_backup /f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment = pathlib.Path.cwd() / caminho_backup /f'{dia_indicador.month}_{dia_indicador.day}_Ranking Diario.xlsx'
mail.Attachments.Add(str(attachment))

mail.Send()
print(f'E-mail da Diretoria enviado!')

E-mail da Diretoria enviado!
